In [6]:
from fastai.tabular.all import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, GroupKFold

In [22]:
df_train =  pd.read_feather('train_24cols.feather')

## TODO: fill with 0 or median depending on the column

In [23]:
df_train = df_train.fillna(0)

In [24]:
kfold = GroupKFold()


In [25]:
df_train.stock_id = df_train.stock_id.astype('category')

In [26]:
cont_nn,cat_nn = cont_cat_split(df_train, max_card=9000, dep_var='target')

In [27]:
cat_nn.remove('time_id'), cat_nn.remove('row_id')

(None, None)

In [28]:
def train_fold(trn_idx, val_idx, fname):
    

    procs_nn = [Categorify, Normalize]
    to_nn = TabularPandas(df_train, procs_nn, cat_nn, cont_nn,
                          splits=[list(trn_idx), list(val_idx)], y_names='target')

    dls = to_nn.dataloaders(1024)
    learn = tabular_learner(dls, y_range=(0,.1), layers=[400,100,500],config={'lin_first':False, 'embed_p':.2, 'ps':[.1,.1,0]},
                        n_out=1, loss_func=rmspe_loss, metrics=AccumMetric(rmspe),
                        #cbs=[EarlyStoppingCallback(monitor='rmspe', comp=np.less, patience=20),
                        #    SaveModelCallback(monitor='rmspe', comp=np.less, fname=fname)]
                           )
    learn.fit_one_cycle(50, 3e-3)
    learn.save(fname)
    return learn.recorder.metrics[0].value

In [29]:
def rmspe(preds, targs):
    x = (targs-preds)/targs
    return (x**2).mean().sqrt()

In [30]:
def rmspe_loss(preds, targs):
    x = (targs-preds)/targs
    return (x**2).mean().sqrt()

In [31]:
res =[]
for trn_idx, val_idx in kfold.split(df_train, groups = df_train.time_id):
    res.append(train_fold(trn_idx, val_idx, f'kfold_{len(res)}'))
    print('fold', len(res), res[-1])
np.mean(res),res

epoch,train_loss,valid_loss,rmspe,time
0,0.358905,0.277361,0.280032,00:02
1,0.269003,0.248247,0.251132,00:02
2,0.257436,0.243346,0.245379,00:02
3,0.254965,0.244841,0.247197,00:02
4,0.249115,0.246860,0.249214,00:02
5,0.257739,0.247060,0.250124,00:02
6,0.256525,0.246878,0.248914,00:02
7,0.250380,0.241097,0.243155,00:02
8,0.250429,0.239995,0.242077,00:02
9,0.248592,0.251928,0.253722,00:02


fold 1 TensorBase(0.2277)


epoch,train_loss,valid_loss,rmspe,time
0,0.374334,0.279297,0.282286,00:02
1,0.271736,0.244819,0.247075,00:02
2,0.263501,0.241644,0.243439,00:02
3,0.257056,0.250093,0.253591,00:02
4,0.255263,0.241566,0.243990,00:02
5,0.258067,0.244886,0.247178,00:02
6,0.252627,0.238120,0.240670,00:02
7,0.250784,0.238947,0.241375,00:02
8,0.250879,0.252618,0.254866,00:02
9,0.247430,0.242934,0.245617,00:02


fold 2 TensorBase(0.2295)


epoch,train_loss,valid_loss,rmspe,time
0,0.378003,0.282440,0.284362,00:02
1,0.272975,0.244952,0.246958,00:02
2,0.270704,0.239511,0.244319,00:02
3,0.259802,0.242979,0.245126,00:02
4,0.260133,0.251798,0.256367,00:02
5,0.256339,0.229929,0.231740,00:02
6,0.255192,0.230970,0.232866,00:02
7,0.249825,0.236607,0.238371,00:02
8,0.255036,0.254460,0.256163,00:02
9,0.255714,0.230495,0.233165,00:02


...

epoch,train_loss,valid_loss,rmspe,time
0,0.375896,0.270064,0.271749,00:02
1,0.266414,0.232487,0.234438,00:02
2,0.267216,0.231502,0.235188,00:02
3,0.257103,0.246371,0.251374,00:02
4,0.258117,0.228177,0.232222,00:02
5,0.258440,0.239187,0.241553,00:02
6,0.255694,0.241298,0.244655,00:02
7,0.254465,0.241211,0.243463,00:02
8,0.259359,0.231276,0.234336,00:02
9,0.253976,0.237031,0.239184,00:02


fold 4 TensorBase(0.2184)


epoch,train_loss,valid_loss,rmspe,time
0,0.368547,0.249640,0.251694,00:03
1,0.269137,0.233144,0.235097,00:03
2,0.258365,0.229692,0.231574,00:03
3,0.262150,0.227138,0.228869,00:03
4,0.255798,0.226708,0.228973,00:03
5,0.268644,0.229668,0.234841,00:03
6,0.252148,0.229482,0.231478,00:03
7,0.272673,0.232890,0.234944,00:04
8,0.254604,0.234408,0.236605,00:03
9,0.258461,0.237483,0.239358,00:04


fold 5 TensorBase(0.2179)


(0.22281937,
 [TensorBase(0.2277),
  TensorBase(0.2295),
  TensorBase(0.2206),
  TensorBase(0.2184),
  TensorBase(0.2179)])